In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
url = 'https://www.thegioididong.com/laptop'
driver.get(url)

# Đợi cho các phần tử được tải đầy đủ trong một khoảng thời gian nhất định (10 giây)
driver.implicitly_wait(10)

def is_view_more_button_displayed():
    try:
        view_more_button = driver.find_element(By.CSS_SELECTOR, '.view-more a')
        return view_more_button.is_displayed()
    except:
        return False

# Vòng lặp để click vào nút "Xem thêm" cho đến khi nó không còn hiển thị
while is_view_more_button_displayed():
    element = driver.find_element(By.CSS_SELECTOR, '.view-more a')
    driver.execute_script("arguments[0].click();", element)

# Lấy đường link sản phẩm từ các thẻ <a> trong <li> có class "item"
product_links = {link.get_attribute('href') for link in driver.find_elements(By.CSS_SELECTOR, 'li.item a') if 'javascript:void(0)' not in link.get_attribute('href') and '#' not in link.get_attribute('href') }
for link in product_links:
        print(link)
# In ra tất cả đường link sản phẩm

c = len(product_links)
print(c)
# Đóng trình duyệt
driver.quit()

https://www.thegioididong.com/laptop/lenovo-ideapad-slim-3-15iah8-i5-83er00evn
https://www.thegioididong.com/laptop/hp-omen-16-n0087ax-r7-7c0t7pa
https://www.thegioididong.com/laptop/dell-inspiron-15-3530-i7-n3530i716w1
https://www.thegioididong.com/laptop/asus-vivobook-15-oled-a1505va-i9-l1386w
https://www.thegioididong.com/laptop/dell-inspiron-15-3520-i5-25p231
https://www.thegioididong.com/laptop/asus-gaming-rog-strix-scar-18-g834jy-i9-n6039w
https://www.thegioididong.com/laptop/hp-envy-16-h0207tx-i7-7c0t4pa
https://www.thegioididong.com/laptop/dell-inspiron-16-5630-i5-71020244
https://www.thegioididong.com/laptop/asus-tuf-gaming-f15-fx507zc4-i5-hn229w
https://www.thegioididong.com/laptop/asus-tuf-gaming-a15-fa507nv-r7-lp031w
https://www.thegioididong.com/laptop/asus-tuf-gaming-a15-fa507nu-r5-lp105w
https://www.thegioididong.com/laptop/hp-pavilion-x360-14-ek1047tu-i7-80r25pa
https://www.thegioididong.com/laptop/acer-aspire-3-a315-58-54xf-i5-nxam0sv007
https://www.thegioididong.com/l

In [7]:
from bs4 import BeautifulSoup

def get_product_info(product_links):
    driver.get(product_links)

    driver.implicitly_wait(10)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    #TÊN SẢN PHẨM
    try:
        product_name = soup.find('h1').text.strip()
    except:
        product_name = ''

    #URL SẢN PHẨM
    try:
        product_url = soup.find('meta', {'property': 'og:url'}).get('content')
    except:
        product_url = ''

    #GIÁ VÀ KHUYẾN MÃI 
    try:
        discounted_price = soup.find('p', class_ = 'box-price-present').text.strip().replace('*', '')
    except:
        discounted_price = ''
    try:
        original_price = soup.find('p', class_ = 'box-price-old').text.strip()
    except:
        original_price = discounted_price
    try:
        discount_percentage = soup.find('p', class_ = 'box-price-percent').text.strip().replace('(', '').replace(')', '').replace('-','')
    except:
        discount_percentage = ''
    
    #TRẢ GÓP
    try:
        installment_payment = soup.find('span', class_ = 'label label--black').text.strip()
    except:
        installment_payment = ''
    
    #KHUYẾN MÃI ĐI KÈM
    gift_promo_elements = soup.find_all('div', class_='divb-right')

    # Tạo một từ điển để lưu các giá trị
    promotions = []

    try:
        for i, gift_promo_element in enumerate(gift_promo_elements, start=1):
        # Trích xuất thông tin từ mỗi phần tử và thêm vào từ điển
            variable_name = f"Promotion__{i}"
            promotion_text = gift_promo_element.find('p').text.strip()
            promotion_text_filter = promotion_text.replace('(Click xem chi tiết)','').replace('(Xem chi tiết tại đây)','')
            promotions.append((variable_name, promotion_text_filter))
    except AttributeError:
        pass

    product_price = {
        'product_name': product_name,
        'discounted_price': discounted_price,
        'original_price': original_price,
        'discount_percentage': discount_percentage,
        'installment_payment' : installment_payment,
        'shop_brand': 'Thế Giới Di Động'
    }
    
    max_promotion = 5
    for i in range(1, max_promotion + 1):
        key = f'promotion_{i}'
        # Nếu i vượt quá số lượng thực sự của khuyến mại, thiết lập giá trị là None
        if i <= len(promotions):
            value = promotions[i-1][1]
        else:
            value = ''
        product_price[key] = value

    product_price['product_url'] = product_url

    return product_price

# Mở trình duyệt
driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duyệt qua các đường link sản phẩm và lấy thông tin
data = []
for product_link in product_links:
    product_price = get_product_info(product_link)
    # if product_price is None:
    #     continue
    data.append(product_price)
    print(product_price)

driver.quit()

{'product_name': 'Laptop Lenovo Ideapad Slim 3 15IAH8 i5 12450H/16GB/512GB/Win11 (83ER000EVN)', 'discounted_price': '15.290.000₫', 'original_price': '15.490.000₫', 'discount_percentage': '', 'installment_payment': '', 'shop_brand': 'Thế Giới Di Động', 'promotion_1': 'Giảm thêm 300.000đ cho Học sinh - Sinh viên (chỉ áp dụng tại siêu thị) ', 'promotion_2': 'Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.000đ) khi thanh toán qua VNPAY-QR, áp dụng cho đơn hàng từ 3 Triệu ', 'promotion_3': '', 'promotion_4': '', 'promotion_5': '', 'product_url': 'https://www.thegioididong.com/laptop/lenovo-ideapad-slim-3-15iah8-i5-83er00evn'}
{'product_name': 'Laptop HP Omen 16 n0087AX R7 6800H/16GB/1TB SSD/6GB RTX3060/165Hz/Win11 (7C0T7PA)', 'discounted_price': '37.790.000₫ ', 'original_price': '50.890.000₫', 'discount_percentage': '25%', 'installment_payment': 'Trả góp 0%', 'shop_brand': 'Thế Giới Di Động', 'promotion_1': 'Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.000đ) khi thanh toán qua VNPAY-QR, áp dụng

In [8]:
import pandas as pd
df = pd.DataFrame(data)
df

,product_name,discounted_price,original_price,discount_percentage,installment_payment,shop_brand,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,product_url
0,Laptop Lenovo Ideapad Slim 3 15IAH8 i5 12450H/...,15.290.000₫,15.490.000₫,,,Thế Giới Di Động,Giảm thêm 300.000đ cho Học sinh - Sinh viên (c...,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,https://www.thegioididong.com/laptop/lenovo-id...
1,Laptop HP Omen 16 n0087AX R7 6800H/16GB/1TB SS...,37.790.000₫,50.890.000₫,25%,Trả góp 0%,Thế Giới Di Động,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,,https://www.thegioididong.com/laptop/hp-omen-1...
2,Laptop Dell Inspiron 15 3530 i7 1355U/16GB/512...,25.290.000₫,25.290.000₫,,,Thế Giới Di Động,Giảm thêm 400.000đ cho Học sinh - Sinh viên (c...,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,https://www.thegioididong.com/laptop/dell-insp...
3,Laptop Asus Vivobook 15 OLED A1505VA i9 13900H...,23.490.000₫,25.990.000₫,9%,Trả góp 0% - Trả trước 10%,Thế Giới Di Động,Giảm thêm 500.000đ cho Học sinh - Sinh viên (c...,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,https://www.thegioididong.com/laptop/asus-vivo...
4,Laptop Dell Inspiron 15 3520 i5 1235U/16GB/512...,15.490.000₫,16.690.000₫,7%,,Thế Giới Di Động,Giảm thêm 300.000đ cho Học sinh - Sinh viên (c...,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,https://www.thegioididong.com/laptop/dell-insp...
...,...,...,...,...,...,...,...,...,...,...,...,...
265,Laptop HP Pavilion 15 eg3035TX i7 1355U/16GB/5...,23.690.000₫,26.590.000₫,10%,Trả góp 0% - Trả trước 10%,Thế Giới Di Động,Giảm thêm 400.000đ cho Học sinh - Sinh viên (c...,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,https://www.thegioididong.com/laptop/hp-pavili...
266,Laptop Dell Vostro 14 3430 i5 1335U/8GB/512GB/...,19.390.000₫,19.990.000₫,,,Thế Giới Di Động,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,,https://www.thegioididong.com/laptop/dell-vost...
267,Laptop HP Pavilion 15 eg3095TU i5 1335U/8GB/51...,17.790.000₫,19.990.000₫,11%,Trả góp 0% - Trả trước 10%,Thế Giới Di Động,Giảm thêm 300.000đ cho Học sinh - Sinh viên (c...,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,,https://www.thegioididong.com/laptop/hp-pavili...
268,Laptop Acer Aspire 7 Gaming A715 43G R8GA R5 5...,15.990.000₫,21.990.000₫,27%,Trả góp 0% - Trả trước 10%,Thế Giới Di Động,Balo Acer Predator Gaming (Hết quà hoàn tiền 4...,Giảm thêm 300.000đ cho Học sinh - Sinh viên (c...,Nhập mã VNPAYTGDD2 giảm ngay 1% (tối đa 200.00...,,,https://www.thegioididong.com/laptop/acer-aspi...


In [9]:
df.to_excel('thegioididong_price.xlsx', index=False)